In [6]:
conda install mlxtend

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - mlxtend

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.


In [3]:
# 데이터 구성: Series, DataFrame
import pandas as pd

# 행렬 연산
import numpy as np

# 데이터 전처리: 항목 값에 대한 index 생성
from mlxtend.preprocessing import TransactionEncoder

# 지지도 계산
from mlxtend.frequent_patterns import apriori

# 연관 규칙
from mlxtend.frequent_patterns import association_rules

### step 1 데이터 구성하기
- 데이터 생성

In [5]:
df_raw = [["소주", "콜라", "맥주"],
           ["소주", "콜라", "와인"],
           ["소주", "주스"],
           ["콜라", "맥주"],
           ["소주", "콜라", "맥주", "와인"],
           ["주스"]]
df_raw

[['소주', '콜라', '맥주'],
 ['소주', '콜라', '와인'],
 ['소주', '주스'],
 ['콜라', '맥주'],
 ['소주', '콜라', '맥주', '와인'],
 ['주스']]

- 데이터 전처리

In [10]:
# Items 중 index 0버이 갖고 있으면 True, 아니면 False
enc = TransactionEncoder()
df_raw_enc = enc.fit_transform(X = df_raw)
# 위의 함수는 데이터를 행렬의 형태로 반환 pandas의 DataFrame의 형태로 반환
df_asso = pd.DataFrame(df_raw_enc, columns=enc.columns_)
df_asso.head()

,맥주,소주,와인,주스,콜라
0,True,True,False,False,True
1,False,True,True,False,True
2,False,True,False,True,False
3,True,False,False,False,True
4,True,True,True,False,True


### step 2 라이브러리 불러오기
- 규칙 척도 및 임계값(지지도 50% 이상)

In [11]:
# 최소 지지도
min_support = 0.5

# 지지도 계산
df_freq = apriori(df_asso, min_support=0.5, use_colnames=True)
df_freq.round(3)

,support,itemsets
0,0.500,(맥주)
1,0.667,(소주)
2,0.667,(콜라)
3,0.500,"(콜라, 맥주)"
4,0.500,"(콜라, 소주)"


- 지지도, 신뢰도, 향상도 산출
* 기준: 지지도(support) / 신뢰도(confidence) / 향상도(lift), 임계값: 기준 metric의 최소값

In [12]:
# 기준 = 신뢰도, 최소 신뢰도 = 0.5
df_asso_rule = association_rules(df_freq, metric="confidence", min_threshold=0.5)
df_asso_rule.round(3)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(콜라),(맥주),0.667,0.500,0.5,0.75,1.500,0.167,2.000
1,(맥주),(콜라),0.500,0.667,0.5,1.00,1.500,0.167,inf
2,(콜라),(소주),0.667,0.667,0.5,0.75,1.125,0.056,1.333
3,(소주),(콜라),0.667,0.667,0.5,0.75,1.125,0.056,1.333


### step 3 결과 해석 및 결론 도출